# Rare Earth Elements Patent Co-occurrence Analysis
## Enhanced with Claude Code AI Capabilities

**Original Analysis**: Riccardo Priore, Centro Patlib – Area Science Park, Trieste

**AI Enhancement Demo**: Live Claude Code demonstration

---

## Background
This notebook analyzes **Rare Earth Elements (REE)** patents using the progression:
1. **Espacenet Search** → Complex query for REE + recycling patents
2. **PATSTAT Analysis** → Patent families, IPC co-occurrence, citations
3. **TIP Enhancement** → Advanced analytics and visualization
4. **🚀 Claude Code AI** → Market correlation, predictive insights, automated reports

### Original Espacenet Search Strategy:
```
(((ctxt=("rare " prox/distance<3 "earth") AND ctxt=("earth" prox/distance<3 "element")) 
OR ctxt=("rare " prox/distance<3 "metal") OR ctxt=("rare " prox/distance<3 "oxide") 
OR ctxt=("light " prox/distance<3 "REE") OR ctxt=("heavy " prox/distance<3 "REE")) 
OR ctxt any "REE" OR ctxt any "lanthan*") AND (ctxt any "recov*" OR ctxt any "recycl*")
```

### Key Results from PATSTAT Analysis:
- **84,905** distinct patent families (keyword-based)
- **567,012** families (classification-based)
- **51,315** IPC co-occurrence patterns (2010-2022)
- Geographic citation analysis across countries

## 1. Setup and Data Loading
*Claude Enhancement Target: Add market data integration and advanced error handling*

In [25]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
warnings.filterwarnings('ignore')

# Set display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Libraries imported successfully!")
print(f"Analysis started at: {datetime.now()}")

# PATSTAT imports with comprehensive error handling
PATSTAT_AVAILABLE = False
PATSTAT_CONNECTED = False

try:
    from epo.tipdata.patstat import PatstatClient
    from epo.tipdata.patstat.database.models import (
        TLS201_APPLN, TLS202_APPLN_TITLE, TLS203_APPLN_ABSTR, 
        TLS209_APPLN_IPC, TLS224_APPLN_CPC, TLS212_CITATION
    )
    from sqlalchemy import func, and_, or_
    from sqlalchemy.orm import sessionmaker, aliased
    
    PATSTAT_AVAILABLE = True
    print("✅ PATSTAT libraries imported successfully")
    
    # Initialize PATSTAT client
    environment = 'PROD'  # Change 'TEST' to 'PROD' for full dataset
    
    print(f"Connecting to PATSTAT {environment} environment...")
    patstat = PatstatClient(env=environment)
    db = patstat.orm()
    
    print(f"✅ Connected to PATSTAT {environment} environment")
    print(f"Database engine: {db.bind}")
    
    # Test table access
    try:
        test_result = db.query(TLS201_APPLN.docdb_family_id).limit(1).first()
        PATSTAT_CONNECTED = True
        print("✅ Table access test successful")
    except Exception as table_error:
        print(f"❌ Table access failed: {table_error}")
        print("⚠️  Issue: BigQuery cannot locate PATSTAT tables in the current configuration")
        print("🔄 Will use enhanced demo data that replicates real PATSTAT patterns")
        PATSTAT_CONNECTED = False
    
except Exception as e:
    print(f"❌ PATSTAT setup failed: {e}")
    print("🔄 Running in demo mode with realistic REE patent data")
    PATSTAT_AVAILABLE = False
    PATSTAT_CONNECTED = False

# Analysis status summary
print("\n📊 Analysis Environment Status:")
print(f"   PATSTAT Libraries: {'✅ Available' if PATSTAT_AVAILABLE else '❌ Not Available'}")
print(f"   PATSTAT Connection: {'✅ Connected' if PATSTAT_CONNECTED else '❌ Table Access Issues'}")
print(f"   Analysis Mode: {'Real Database' if PATSTAT_CONNECTED else 'Enhanced Demo Data'}")

print("\n🚀 Ready for Claude Code AI enhancement!")
print(f"Demo time: {datetime.now()}")

Libraries imported successfully!
Analysis started at: 2025-06-24 13:52:07.730276
✅ PATSTAT libraries imported successfully
Connecting to PATSTAT PROD environment...
✅ Connected to PATSTAT PROD environment
Database engine: Engine(bigquery+custom_dialect://p-epo-tip-prj-3a1f/p_epo_tip_euwe4_bqd_patstata)
✅ Table access test successful

📊 Analysis Environment Status:
   PATSTAT Libraries: ✅ Available
   PATSTAT Connection: ✅ Connected
   Analysis Mode: Real Database

🚀 Ready for Claude Code AI enhancement!
Demo time: 2025-06-24 13:52:08.520168


## 2. REE Patent Search Implementation
*Enhancement Target: Add real-time Espacenet API integration*

In [ ]:
# REE Patent Search with Robust PATSTAT Integration
# =================================================

# Riccardo's comprehensive search strategy
ree_keywords = [
    "rare earth element", "light REE", "heavy REE", "rare earth metal",
    "rare earth oxide", "lanthan", "rare earth", "neodymium", "dysprosium",
    "terbium", "europium", "yttrium", "cerium", "lanthanum", "praseodymium"
]

recovery_keywords = ["recov", "recycl", "extract", "separat", "purif"]

# IPC/CPC classification codes from Riccardo's analysis
key_classification_codes = [
    'C22B  19/28', 'C22B  19/30', 'C22B  25/06',  # REE extraction
    'C04B  18/04', 'C04B  18/06', 'C04B  18/08',  # REE ceramics/materials  
    'H01M   6/52', 'H01M  10/54',  # REE batteries
    'C09K  11/01',  # REE phosphors
    'H01J   9/52',  # REE displays
    'Y02W30/52', 'Y02W30/56', 'Y02W30/84',  # Recycling technologies
]

def execute_ree_patent_search():
    """
    Execute REE patent search using best available method
    """
    if PATSTAT_CONNECTED:
        return execute_real_patstat_search()
    else:
        return execute_enhanced_demo_search()

def execute_real_patstat_search():
    """
    Real PATSTAT search using proven working patterns from testing
    """
    try:
        print("🔍 Executing Real PATSTAT REE Patent Search...")
        
        # Step 1: Keywords-based search (WORKING PATTERN from testing)
        # Use focused keywords and recent date range to avoid timestamp issues
        focused_ree_keywords = ["rare earth", "lanthan", "neodymium"]  # Proven working subset
        focused_recovery_keywords = ["recov", "recycl"]  # Proven working subset
        
        subquery_abstracts = (
            db.query(TLS201_APPLN.docdb_family_id, TLS201_APPLN.appln_id, 
                     TLS201_APPLN.appln_filing_date, TLS201_APPLN.appln_nr)
            .join(TLS203_APPLN_ABSTR, TLS203_APPLN_ABSTR.appln_id == TLS201_APPLN.appln_id)
            .filter(
                and_(
                    TLS201_APPLN.appln_filing_date >= '2020-01-01',  # Recent data only - PROVEN WORKING
                    TLS201_APPLN.appln_filing_date <= '2024-12-31',
                    or_(*[TLS203_APPLN_ABSTR.appln_abstract.contains(kw) for kw in focused_ree_keywords]),
                    or_(*[TLS203_APPLN_ABSTR.appln_abstract.contains(rw) for rw in focused_recovery_keywords])
                )
            ).distinct().limit(100)  # Start with manageable limit
        )
        
        keywords_results = subquery_abstracts.all()
        keywords_families = [row.docdb_family_id for row in keywords_results]
        
        print(f"✅ Keywords search: {len(keywords_results)} applications found")
        
        # Step 2: Classification-based search (WORKING PATTERN from testing)
        focused_classification_codes = ['C22B  19/28', 'C22B  19/30', 'C04B  18/04', 'H01M   6/52']
        
        subquery_ipc = (
            db.query(TLS201_APPLN.docdb_family_id, TLS201_APPLN.appln_id,
                     TLS201_APPLN.appln_filing_date, TLS209_APPLN_IPC.ipc_class_symbol)
            .join(TLS209_APPLN_IPC, TLS209_APPLN_IPC.appln_id == TLS201_APPLN.appln_id)
            .filter(
                and_(
                    TLS201_APPLN.appln_filing_date >= '2020-01-01',  # PROVEN WORKING
                    TLS201_APPLN.appln_filing_date <= '2024-12-31',
                    func.substr(TLS209_APPLN_IPC.ipc_class_symbol, 1, 11).in_(focused_classification_codes)
                )
            ).distinct().limit(100)
        )
        
        classification_results = subquery_ipc.all()
        classification_families = [row.docdb_family_id for row in classification_results]
        
        print(f"✅ Classification search: {len(classification_results)} applications found")
        
        # Combine all results
        all_families = list(set(keywords_families + classification_families))
        intersection_families = list(set(keywords_families) & set(classification_families))
        
        print(f"📊 Total unique families: {len(all_families)}")
        print(f"🎯 High-quality intersection: {len(intersection_families)}")
        
        # Build comprehensive dataset using all found families
        if len(all_families) > 0:
            final_query = (
                db.query(TLS201_APPLN.appln_id, TLS201_APPLN.appln_nr, 
                         TLS201_APPLN.appln_filing_date, TLS201_APPLN.docdb_family_id,
                         TLS201_APPLN.earliest_filing_year)
                .filter(TLS201_APPLN.docdb_family_id.in_(all_families))
                .distinct()
            )
            
            final_results = final_query.all()
            df_result = pd.DataFrame(final_results, columns=[
                'appln_id', 'appln_nr', 'appln_filing_date', 'docdb_family_id', 'earliest_filing_year'
            ])
            
            # Add quality indicators
            df_result['search_method'] = 'Real PATSTAT (Focused Keywords + Classification)'
            df_result['quality_score'] = df_result['docdb_family_id'].apply(
                lambda x: 1.0 if x in intersection_families else 0.8
            )
            df_result['filing_year'] = pd.to_datetime(df_result['appln_filing_date']).dt.year
            
            print("✅ Real PATSTAT search successful!")
            print(f"📈 Found {len(df_result)} REE patent applications")
            print(f"🏆 Average quality score: {df_result['quality_score'].mean():.2f}")
            
            return df_result
        else:
            print("⚠️ No results found - switching to demo data")
            return execute_enhanced_demo_search()
        
    except Exception as e:
        print(f"❌ Real PATSTAT search failed: {e}")
        print(f"   Error type: {type(e).__name__}")
        print("🔄 Falling back to enhanced demo data...")
        return execute_enhanced_demo_search()

def execute_enhanced_demo_search():
    """
    Enhanced demo search based on Riccardo's actual findings
    """
    print("📊 Executing Enhanced Demo REE Patent Search...")
    print("🎯 Based on Riccardo's verified PATSTAT analysis results")
    
    # Riccardo's verified results from real PATSTAT analysis
    print("📈 Riccardo's Original Results:")
    print("   • 84,905 families (keyword-based)")
    print("   • 567,012 families (classification-based)") 
    print("   • ~51,315 IPC co-occurrence patterns")
    print("   • Geographic analysis: US patents cited more internationally than Chinese")
    
    # Create realistic demo dataset matching Riccardo's patterns
    np.random.seed(42)  # Reproducible results
    
    # Scale down proportionally for demo (1:1000 ratio)
    n_demo_families = 85  # Represents ~85,000 real families
    
    # Geographic distribution reflecting real REE patent landscape
    countries = ['CN', 'US', 'JP', 'DE', 'KR', 'CA', 'AU', 'FR', 'GB', 'NL']
    # China leads (35%), followed by US (20%), Japan (15%), etc.
    country_weights = [0.35, 0.20, 0.15, 0.08, 0.06, 0.05, 0.04, 0.03, 0.02, 0.02]
    
    # Technology areas from Riccardo's classification analysis  
    tech_areas = ['Metallurgy & Extraction', 'Recycling & Recovery', 'Electronics & Magnetics',
                  'Ceramics & Materials', 'Processing & Separation', 'Other Applications']
    tech_weights = [0.25, 0.20, 0.18, 0.15, 0.12, 0.10]
    
    demo_data = {
        'appln_id': range(1000000, 1000000 + n_demo_families),
        'appln_nr': [f'{np.random.choice(["EP", "US", "CN", "JP"])}{2010 + i//10}{str(i%10000).zfill(6)}' 
                     for i in range(n_demo_families)],
        'docdb_family_id': range(500000, 500000 + n_demo_families),
        'appln_filing_date': pd.date_range('2010-01-01', '2022-12-31', periods=n_demo_families),
        'geographic_origin': np.random.choice(countries, n_demo_families, p=country_weights),
        'technology_area': np.random.choice(tech_areas, n_demo_families, p=tech_weights),
        'search_method': 'Enhanced Demo (Riccardo-based)',
        'quality_score': np.random.uniform(0.85, 1.0, n_demo_families),  # High quality
        'market_relevance': np.random.uniform(0.7, 1.0, n_demo_families)
    }
    
    df_demo = pd.DataFrame(demo_data)
    df_demo['filing_year'] = pd.to_datetime(df_demo['appln_filing_date']).dt.year
    df_demo['earliest_filing_year'] = df_demo['filing_year']  # For compatibility
    
    print("✅ Enhanced demo dataset created")
    print(f"📊 Demo families: {len(df_demo):,} (represents ~{len(df_demo)*1000:,} real families)")
    print(f"🌍 Geographic coverage: {df_demo['geographic_origin'].nunique()} countries")
    print(f"🏷️ Technology areas: {df_demo['technology_area'].nunique()} domains")
    print(f"📅 Temporal range: {df_demo['filing_year'].min()}-{df_demo['filing_year'].max()}")
    
    return df_demo

# Execute the REE patent search
print("🚀 Starting REE Patent Search")
print("="*50)

high_quality_ree = execute_ree_patent_search()

print("\n✅ REE Patent Search Complete")
print(f"📊 Dataset: {len(high_quality_ree):,} patent families")
print(f"🎯 Search method: {high_quality_ree['search_method'].iloc[0] if len(high_quality_ree) > 0 else 'None'}")
if 'quality_score' in high_quality_ree.columns:
    print(f"🏆 Average quality score: {high_quality_ree['quality_score'].mean():.2f}")

# Display sample results
if len(high_quality_ree) > 0:
    print("\n📋 Sample Dataset:")
    display_cols = ['appln_nr', 'filing_year']
    if 'geographic_origin' in high_quality_ree.columns:
        display_cols.append('geographic_origin')
    if 'technology_area' in high_quality_ree.columns:
        display_cols.append('technology_area')
    
    sample_data = high_quality_ree[display_cols].head()
    print(sample_data.to_string(index=False))

print("\n🚀 Ready for co-occurrence analysis and Claude Code AI enhancement")

## 3. IPC Co-occurrence Analysis
*Enhancement Target: Add dynamic co-occurrence analysis and trend prediction*

In [ ]:
# Generate demo co-occurrence patterns based on Riccardo's analysis
def get_demo_cooccurrence_patterns():
    """
    Fallback demo co-occurrence patterns based on Riccardo's analysis
    """
    print("📊 Using Demo Co-occurrence Patterns (Riccardo's 51,315 patterns)")
    
    np.random.seed(42)  # Reproducible results
    
    # Key IPC co-occurrence patterns from REE analysis
    key_ipc_pairs = [
        ('C22B   3', 'C07D 257'),  # Metallurgy + Organic chemistry
        ('C22B  59', 'C22B   7'),  # Different metallurgy processes
        ('H01M  10', 'H10N  35'),  # Battery technologies
        ('C04B  18', 'C09K  11'),  # Ceramic materials + Luminescent materials
        ('B03C   1', 'C22B  59'),  # Magnetic separation + Metallurgy
        ('H01F  13', 'H05B   6'),  # Magnets + Induction heating
    ]
    
    # Create sample dataset matching Riccardo's structure
    sample_data = []
    for i, (ipc1, ipc2) in enumerate(key_ipc_pairs * 50):  # Expand dataset
        for year in range(2012, 2023):
            if np.random.random() > 0.3:  # 70% chance of data point
                count = np.random.poisson(5) + 1  # Average ~5 families per combination
                sample_data.append({
                    'IPC_1': ipc1,
                    'IPC_2': ipc2, 
                    'earliest_filing_year': year,
                    'count_of_families': count
                })
    
    return pd.DataFrame(sample_data)

# Execute co-occurrence analysis
print("🚀 Starting IPC Co-occurrence Analysis")
print("="*50)

df_cooccurrence = get_demo_cooccurrence_patterns()

# Display results
if len(df_cooccurrence) > 0:
    # Display top patterns
    top_patterns = df_cooccurrence.groupby(['IPC_1', 'IPC_2'])['count_of_families'].sum().sort_values(ascending=False).head(10)
    print("\n🏆 Top IPC Co-occurrence Patterns:")
    for (ipc1, ipc2), count in top_patterns.items():
        print(f"   {ipc1} ↔ {ipc2}: {count} families")
    
    # Technology area mapping
    def get_technology_area(ipc_code):
        """Map IPC codes to technology areas"""
        if ipc_code.startswith('C22B'):
            return 'Metallurgy & Extraction'
        elif ipc_code.startswith('H01'):
            return 'Electronics & Energy'
        elif ipc_code.startswith('C04B') or ipc_code.startswith('C09K'):
            return 'Materials & Ceramics'
        elif ipc_code.startswith('B'):
            return 'Processing & Separation'
        else:
            return 'Other Applications'
    
    # Add technology areas
    df_cooccurrence['tech_area_1'] = df_cooccurrence['IPC_1'].apply(get_technology_area)
    df_cooccurrence['tech_area_2'] = df_cooccurrence['IPC_2'].apply(get_technology_area)
    
    # Cross-technology analysis
    cross_tech = df_cooccurrence[df_cooccurrence['tech_area_1'] != df_cooccurrence['tech_area_2']]
    print(f"\n🔄 Cross-Technology Convergence: {len(cross_tech)} patterns ({len(cross_tech)/len(df_cooccurrence)*100:.1f}%)")
    
else:
    print("⚠️  No co-occurrence patterns found")

print("\n🚀 Enhanced Claude Code Analysis Targets:")
print("   • Real-time pattern detection and trend prediction")
print("   • Technology convergence mapping with market data")
print("   • Supply chain vulnerability assessment")
print("   • EU Green Deal alignment analysis")
print("   • Investment opportunity identification")

## 4. Market Data Integration Point
*🚀 Claude Enhancement: Correlate patents with JRC market data*

In [ ]:
# Market Data Integration Opportunity
print("📊 JRC Market Data Integration Opportunity:")
print("   Available: Rare_Earth_Metals_Market.pdf → Excel data")
print("   Available: Rare_Earth_Metals_Recycling_Market.pdf → Excel data")
print("")
print("🎯 Claude Enhancement Goals:")
print("   • Correlate patent filing trends with market prices")
print("   • Identify patent-market timing patterns")
print("   • Predict technology adoption based on market signals")
print("   • Map supply disruptions to innovation responses")
print("")
print("📈 Expected Correlations to Discover:")
print("   • 2010-2011 REE crisis → Patent filing surge")
print("   • Wind energy growth → Magnet technology patents")
print("   • EV adoption → Battery REE recycling patents")
print("   • Trade tensions → Alternative technology development")

# Sample market indicators (to be replaced with real JRC data)
market_events = {
    2010: "REE Crisis Begins",
    2011: "Price Peak (Neodymium $500/kg)", 
    2014: "Market Stabilization",
    2017: "EV Market Acceleration",
    2019: "Trade War Impact",
    2020: "COVID Supply Disruption",
    2022: "Green Deal Implementation"
}

print("\n🗓️  Key Market Events for Patent Correlation:")
for year, event in market_events.items():
    print(f"   {year}: {event}")

print("\n🚀 Ready for live Claude Code enhancement!")

---

## 🚀 Live Claude Code Enhancement Roadmap

### Phase 1: Market Data Integration (10 min)
- [ ] Load and parse JRC rare earth market data
- [ ] Create patent-market correlation analysis
- [ ] Identify market-driven innovation patterns
- [ ] Generate supply-demand vs. patent activity charts

### Phase 2: AI-Powered Insights (10 min)
- [ ] Technology trend prediction (2024-2030)
- [ ] Supply chain vulnerability mapping
- [ ] Innovation gap analysis
- [ ] Competitive intelligence automation

### Phase 3: Advanced Visualization (10 min)
- [ ] Interactive geographic patent-market dashboard
- [ ] Time-series correlation plots
- [ ] Technology convergence network analysis
- [ ] Policy impact visualization

### Phase 4: Automated Reporting (10 min)
- [ ] Executive summary generation
- [ ] Policy maker briefing documents
- [ ] Investment opportunity reports
- [ ] Supply chain risk assessments

---

## Value Proposition: Espacenet → PATSTAT → TIP → Claude Code AI

**Riccardo's Foundation**: Comprehensive REE patent landscape using professional tools

**Claude Code Enhancement**: AI-powered insights, market correlation, predictive analytics

**Result**: From static analysis to dynamic intelligence for critical raw materials strategy

---

*This notebook demonstrates the full evolution from basic patent searching to AI-enhanced strategic intelligence for critical materials like Rare Earth Elements*